<a href="https://colab.research.google.com/github/pysr1/medium/blob/master/tfidf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np
import seaborn as sns 
import matplotlib.pyplot as plt
from nltk.corpus import stopwords 
import nltk
import string
nltk.download('stopwords')


medium = 'https://raw.githubusercontent.com/rfordatascience/tidytuesday/master/data/2018/2018-12-04/medium_datasci.csv'
medium_df = pd.read_csv(medium).drop('x1', 1)    
medium_df = medium_df[~medium_df['title'].isna()]
medium_df['subtitle'] = medium_df['subtitle'].fillna("")
medium_df['publication'] = medium_df['publication'].fillna("")
medium_df['text'] = medium_df['title'] + " " + medium_df['subtitle'] + " " + medium_df['publication']
medium_df['words'] = medium_df['text'].str.strip().str.split('[\W_]+')


medium_df.head()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


,title,subtitle,image,author,publication,year,month,day,reading_time,claps,url,author_url,tag_ai,tag_artificial_intelligence,tag_big_data,tag_data,tag_data_science,tag_data_visualization,tag_deep_learning,tag_machine_learning,text,words
0,Online Animation: Mixamo vs Norah AI,Online animations tools provide game designers...,1,Emma Laurent,,2017,8,1,5,12.0,https://medium.com/@laurentemma/online-animati...,https://medium.com/@laurentemma,1,0,0,0,0,0,0,0,Online Animation: Mixamo vs Norah AI Online an...,"[Online, Animation, Mixamo, vs, Norah, AI, Onl..."
1,A.I. ?,,0,Sanparith Marukatat,,2017,8,1,2,11.0,https://medium.com/@sanparithmarukatat/a-i-%E0...,https://medium.com/@sanparithmarukatat,1,0,0,0,0,0,0,0,A.I. ?,"[A, I, ]"
2,"Futures of AI, Friendly AI?",From Physics of the Future: How Science Will S...,1,Z,,2017,8,1,3,1.0,https://medium.com/@otherside914/futures-of-ai...,https://medium.com/@otherside914,1,0,0,0,0,0,0,0,"Futures of AI, Friendly AI? From Physics of th...","[Futures, of, AI, Friendly, AI, From, Physics,..."
3,The Value Generated through AI led Mindfulness,A true product co-created by the Millennials,1,Emiko Sawaguchi,MILLENNIALSTIMES,2017,8,1,5,4.0,https://millennials-times.com/creativeflower-e...,https://millennials-times.com/@emikosawaguchi,1,0,0,0,0,0,0,0,The Value Generated through AI led Mindfulness...,"[The, Value, Generated, through, AI, led, Mind..."
4,"Os Melhores Links de JulhoCmeras Vintage, Goog...",mas pera a?,1,Giovani Ferreira,NEW ORDER,2017,8,1,3,56.0,https://medium.com/neworder/os-melhores-links-...,https://medium.com/@gferreira,1,0,0,0,0,0,0,0,"Os Melhores Links de JulhoCmeras Vintage, Goog...","[Os, Melhores, Links, de, JulhoCmeras, Vintage..."


In [0]:
publications = medium_df.groupby('publication').median()[['claps']].sort_values('claps', ascending = False)
publications['count'] = medium_df.groupby('publication').size()
publications.head(20).style.background_gradient()

,claps,count
publication,,
TomYum,23000,1
Product Hunt,13000,1
IOTA,9420,2
Ubex,8200,15
sarahcpr,7900,1
Words That Matter,5800,1
hello JS,5300,1
Unchained Capital Blog,4900,2
Bradfield,4500,1


In [0]:
publications['metric'] = publications['claps'] * publications['count']
tops = publications.sort_values('metric', ascending = False).head(11).index.to_list()

In [0]:
tops.remove("")
medium_df = medium_df[medium_df['publication'].isin(tops)]
medium_df.head(10)

,title,subtitle,image,author,publication,year,month,day,reading_time,claps,url,author_url,tag_ai,tag_artificial_intelligence,tag_big_data,tag_data,tag_data_science,tag_data_visualization,tag_deep_learning,tag_machine_learning,text,words
5,The Future of AI: Redefining How We Imagine,Original article posted on FICO blogs,1,Syed Sadat Nazrul,Towards Data Science,2017,8,1,6,103.0,https://towardsdatascience.com/the-future-of-a...,https://towardsdatascience.com/@sadatnazrul,1,0,0,0,0,0,0,0,The Future of AI: Redefining How We Imagine Or...,"[The, Future, of, AI, Redefining, How, We, Ima..."
60,Artificial Intelligence and the Rise of Econom...,,1,Abhinav Suri,Towards Data Science,2017,8,9,16,577.0,https://towardsdatascience.com/artificial-inte...,https://towardsdatascience.com/@abhisuri97,1,0,0,0,0,0,0,0,Artificial Intelligence and the Rise of Econom...,"[Artificial, Intelligence, and, the, Rise, of,..."
102,AI + eSports: Thoughts on the Future,eSports is the new frontier for AI companies l...,1,Arkin Dharawat,Towards Data Science,2017,8,15,4,22.0,https://towardsdatascience.com/ai-esports-thou...,https://towardsdatascience.com/@arkindharawat,1,0,0,0,0,0,0,0,AI + eSports: Thoughts on the Future eSports i...,"[AI, eSports, Thoughts, on, the, Future, eSpor..."
172,Will Your Work Be (Mis) Judged By Machines?,,1,David Smooke,Hacker Noon,2017,8,24,3,198.0,https://hackernoon.com/will-your-work-be-mis-j...,https://hackernoon.com/@DavidSmooke,1,0,0,0,0,0,0,0,Will Your Work Be (Mis) Judged By Machines? H...,"[Will, Your, Work, Be, Mis, Judged, By, Machin..."
290,Logistic Regression,,1,Parag Radke,Towards Data Science,2017,9,14,3,90.0,https://towardsdatascience.com/logistic-regres...,https://towardsdatascience.com/@paragradke,1,0,0,0,0,0,0,0,Logistic Regression Towards Data Science,"[Logistic, Regression, Towards, Data, Science]"
296,Video Interview with grocery shoppers. Would y...,Amazon plans to launch cashier-less grocery st...,0,Flora Q.,Hacker Noon,2017,9,14,2,7.0,https://hackernoon.com/video-interview-with-gr...,https://hackernoon.com/@folalaflora,1,0,0,0,0,0,0,0,Video Interview with grocery shoppers. Would y...,"[Video, Interview, with, grocery, shoppers, Wo..."
512,Driverless cars wont play chicken,A question of machines and ethics,1,babulous,Hacker Noon,2017,10,12,2,16.0,https://hackernoon.com/driverless-cars-that-do...,https://hackernoon.com/@babulous,1,0,0,0,0,0,0,0,Driverless cars wont play chicken A question o...,"[Driverless, cars, wont, play, chicken, A, que..."
666,Robot Granted Citizenship in Saudi Arabia,"Hanson Robotics, based out of Hong Kong, creat...",0,Sean McCormick,Hacker Noon,2017,10,31,2,38.0,https://hackernoon.com/robot-granted-citizensh...,https://hackernoon.com/@sean.mccormick.,1,0,0,0,0,0,0,0,Robot Granted Citizenship in Saudi Arabia Hans...,"[Robot, Granted, Citizenship, in, Saudi, Arabi..."
726,AI in Medicine: A Physicians Perspective,It is funny how much I want to learn but just ...,1,Tanisha Bassan,Hacker Noon,2017,11,6,6,13.0,https://hackernoon.com/ai-in-medicine-a-physic...,https://hackernoon.com/@Tanisha.Bassan,1,0,0,0,0,0,0,0,AI in Medicine: A Physicians Perspective It is...,"[AI, in, Medicine, A, Physicians, Perspective,..."
747,The Most Promising Internet of Things trends f...,,1,DashMagazine,Hacker Noon,2017,11,8,5,348.0,https://hackernoon.com/the-most-promising-inte...,https://hackernoon.com/@dashbouquet,1,0,0,0,0,0,0,0,The Most Promising Internet of Things trends f...,"[The, Most, Promising, Internet, of, Things, t..."


In [0]:
STOP_WORDS = stopwords.words('english')
rows = list()
for row in medium_df[['publication', 'words']].iterrows():
    r = row[1]
    for word in r.words:
        rows.append((r.publication, word))


words = pd.DataFrame(rows, columns=['publication', 'word'])
words['word'] = words['word'].str.lower()
words = words[~words['word'].isin(STOP_WORDS)]
words = words[words.word.str.len() > 0]

# this should have been done before - remember when rewriting

words.head()

,publication,word
1,Towards Data Science,future
3,Towards Data Science,ai
4,Towards Data Science,redefining
7,Towards Data Science,imagine
8,Towards Data Science,original


In [0]:
counts = words.groupby('publication')\
    .word.value_counts()\
    .to_frame()\
    .rename(columns={'word':'n_w'})
counts.head()

n_w
publication        word          
Creative Analytics analytics   60
                   creative    49
                   lessons     22
                   data        10
                   bi           5

In [0]:
def pretty_plot_top_n(series, top_n=5, index_level=0):
    r = series\
    .groupby(level=index_level)\
    .nlargest(top_n)\
    .reset_index(level=index_level, drop=True)
    return r.to_frame()


pretty_plot_top_n(counts['n_w'])

n_w
publication                word                 
Creative Analytics         analytics          60
                           creative           49
                           lessons            22
                           data               10
                           bi                  5
Hacker Noon                hacker           1174
                           noon             1173
                           ai                296
                           data              282
                           learning          255
Intuition Machine          intuition          93
                           machine            88
                           learning           49
                           deep               45
                           ai                 11
Mybridge for Professionals mybridge           41
                           professionals      41
                           v                  40
                           month              35
                           top                31
SingularityNET             singularitynet     99
                           ai                 35
                           partners           11
                           singularitynets     9
                           decentralized       8
SwipeCrypto                swipecrypto        18
                           swipe              13
                           partnership         6
                           swipes              6
                           update              6
The Startup                startup           266
                           data               56
                           ai                 52
                           intelligence       34
                           artificial         32
Towards Data Science       data             4248
                           science          3484
                           towards          3117
                           learning          878
                           machine           476
Ubex                       ubex               41
                           advertising         5
                           attended            5
                           exchange            5
                           project             5
freeCodeCamp.org           freecodecamp      203
                           org               201
                           data               42
                           learning           36
                           using              22

In [0]:
word_sum = counts.groupby(level=0)\
    .sum()\
    .rename(columns={'n_w': 'n_d'})
word_sum

,n_d
publication,
Creative Analytics,503
Hacker Noon,14894
Intuition Machine,875
Mybridge for Professionals,477
SingularityNET,731
SwipeCrypto,195
The Startup,2723
Towards Data Science,41261
Ubex,189


In [0]:
tf = counts.join(word_sum)
c_d = words.publication.nunique()
tf['tf'] = tf.n_w/tf.n_d

tf.head()


idf = words.groupby('word')\
    .publication\
    .nunique()\
    .to_frame()\
    .rename(columns={'publication':'i_d'})\
    .sort_values('i_d')

idf['idf'] = np.log(c_d/idf.i_d.values)

tf_idf = tf.join(idf)
tf_idf['tf_idf'] = tf_idf.tf * tf_idf.idf


In [0]:
pretty_plot_top_n(tf_idf['tf_idf'])

tf_idf
publication                word                     
Creative Analytics         creative         0.117286
                           analytics        0.060933
                           lessons          0.030317
                           comic            0.015998
                           superhero        0.015998
Hacker Noon                hacker           0.181498
                           noon             0.181344
                           ai               0.007088
                           data             0.006753
                           machine          0.005693
Intuition Machine          intuition        0.127965
                           machine          0.051374
                           deep             0.018343
                           learning         0.012496
                           gary             0.007895
Mybridge for Professionals mybridge         0.197916
                           professionals    0.078759
                           v                0.076838
                           month            0.050860
                           past             0.042141
SingularityNET             singularitynet   0.217968
                           singularitynets  0.028349
                           partners         0.018117
                           ai               0.017077
                           reputation       0.012600
SwipeCrypto                swipecrypto      0.212546
                           swipe            0.153506
                           swipes           0.070849
                           biweekly         0.059041
                           partnership      0.049521
The Startup                startup          0.089509
                           data             0.007335
                           ai               0.006811
                           chatbots         0.006190
                           marketing        0.006190
Towards Data Science       towards          0.090952
                           science          0.058528
                           data             0.036721
                           machine          0.005893
                           learning         0.004748
Ubex                       ubex             0.499503
                           pioneering       0.048732
                           attended         0.042578
                           d10e             0.036549
                           advertising      0.031851
freeCodeCamp.org           freecodecamp     0.219654
                           org              0.113721
                           tensorflow       0.007921
                           data             0.007040
                           js               0.005092